In [8]:
#  Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import pprint

#  Importing API key
from api_keys import bea_api_key

base_url = "https://apps.bea.gov/api/data/"

In [9]:
# BEA CLient for Industry Analysis

class BEAClientIndustry:

  def __init__(self, api_key):
    self.api_key = api_key
    self.base_url = base_url
    self.common_params = {
        'UserID': self.api_key,
        'method': 'GetData',
        'ResultFormat': 'JSON'
    }
    
  def make_request(self, params):
    try:
      full_params = {**self.common_params, **params}
      response = requests.get(self.base_url, params=full_params)
      response.raise_for_status()
      return response.json()
    except requests.exceptions.RequestException as e:
      print(f"API request failed: {e}")
      return None

  def get_gdp_industry_data(self, years=['2018', '2019', '2020', '2021', '2022', '2023']):

    params = {
        'datasetname': 'GDPbyIndustry',
        'Year': ','.join(years),
        'Industry': 'ALL',
        'TableID': '5',  # Value Added by Industry
        'Frequency': 'Q'  
        }
    return self.make_request(params)
  
# Initializing Client
industry_analyzer = BEAClientIndustry(bea_api_key)
print("BEA Client initialized")

BEA Client initialized


In [10]:
def process_gdpIndustry_data(raw_data):
  if not raw_data or 'BEAAPI' not in raw_data:
    print('No Data Received. Invalid Input')
    return None 

  try:
    results = raw_data['BEAAPI']['Results']['Data']
    processed_data = []
    for item in results:
      try:
        value = float(item['DataValue'].replace(',', ''))
      except (ValueError, AttributeError):
        continue
      processed_data.append({
          'Industry % of GDP': value,
          'Industry Description': item['IndustrYDescription'],
          'Industry': item['Industry'],
          'NoteRef': item['NoteRef'],
          'Quarter': item['Quarter'],
          'TableID': item['TableID'],
          'Year': item['Year']
          # 'TableName': item.get('TableName', 'Unknown')
          })
    gdp_by_industry_df = pd.DataFrame(processed_data)

    return gdp_by_industry_df

  except Exception as e:
    print(f"Error processing data: {e}")
    return None